# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2b51e760f5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and Divine Comedy explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some italian poets in terms of place of birth, number of literary works and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q40185`| The Divine Comedy  | node |





Also consider

```
wd:Q40185 ?p ?obj .
```

is the BGP to retrieve all **properties of The Divine Comedy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who wrote The Divine Comedy? (return the IRI and the label of the author)

2. Identify the BGP for poet

3. Identify the BGP for literary movement

4. Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

5. Identify the BGP for literary work

6. Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

7. Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

8. Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label). 

## Task 1
Who wrote The Divine Comedy? (return the IRI and the label of the author)

In [6]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q40185 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2551'), ('name', 'used metre')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de Fran

In [13]:
# [('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')] -> predicate= author: wdt:P50
queryString = """
SELECT DISTINCT ?name
WHERE {
   # using "author" as predicate
   wd:Q40185 wdt:P50 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Dante Alighieri')]
1


Final query for this task

In [15]:
# write your final query
#returning both IRI and Label
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q40185 wdt:P50 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
1


## Task 2
Identify the BGP for poet

In [71]:

queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   
   # Using Dante Alighieri (Q1067) as an example of poets and finding properties
   wd:Q1067 ?p ?obj .
   
   # get the label
   ?p sc:name ?pname .
   FILTER (!isLiteral(?obj))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('pname', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pname', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pname', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('pname', 'depicted by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('pname', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pname', 'ethnic group')]
[('p', 'http

In [52]:
queryString = """
SELECT DISTINCT ?job ?name
WHERE {
   
   #From results: [('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
   wd:Q1067 ?p ?obj ;
            wdt:P106 ?job .
   # get the label
   ?job sc:name ?name .

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('job', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('job', 'http://www.wikidata.org/entity/Q15994177'), ('name', 'political theorist')]
[('job', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('job', 'http://www.wikidata.org/entity/Q12144794'), ('name', 'prosaist')]
[('job', 'http://www.wikidata.org/entity/Q822146'), ('name', 'lyricist')]
[('job', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('job', 'http://www.wikidata.org/entity/Q58968'), ('name', 'intellectual')]
[('job', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
9


Final query for this task

In [148]:
queryString = """
SELECT DISTINCT ?job ?name
WHERE {
   
   #From results: [('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
   wd:Q1067 ?p ?obj ;
            wdt:P106 ?job .
   # get the label
   ?job sc:name ?name .

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('job', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('job', 'http://www.wikidata.org/entity/Q15994177'), ('name', 'political theorist')]
[('job', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('job', 'http://www.wikidata.org/entity/Q12144794'), ('name', 'prosaist')]
[('job', 'http://www.wikidata.org/entity/Q822146'), ('name', 'lyricist')]
[('job', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('job', 'http://www.wikidata.org/entity/Q58968'), ('name', 'intellectual')]
[('job', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
9


## Task 3
Identify the BGP for literary movement

In [97]:

queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   
   # Using Dante Alighieri (Q1067) as an example of poets and finding properties
   wd:Q1067 ?p ?obj .
   
   # get the label
   ?p sc:name ?pname .
   FILTER (!isLiteral(?obj))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('pname', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pname', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pname', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('pname', 'depicted by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('pname', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pname', 'ethnic group')]
[('p', 'http

In [98]:
# from results: [('p', 'http://www.wikidata.org/prop/direct/P135'), ('pname', 'movement')]


queryString = """
SELECT DISTINCT ?m ?name
WHERE {
   
   # using "instance of" property
   wd:Q1067 wdt:P135 ?obj .
   ?obj wdt:P31 ?m .
   
   # get the label
   ?m sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q3326717'), ('name', 'literary movement')]
1


Final query for this task

In [147]:
queryString = """
SELECT DISTINCT ?m ?name
WHERE {
   
   # using "instance of" property
   wd:Q1067 wdt:P135 ?obj .
   ?obj wdt:P31 ?m .
   
   # get the label
   ?m sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q3326717'), ('name', 'literary movement')]
1


## Task 4
Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

In [119]:
#Finding birth characterictics of the author
queryString = """
SELECT DISTINCT ?author ?p ?pname ?o ?oname
WHERE {
   
   wd:Q40185 wdt:P50 ?author .
   ?author ?p ?o .
   
   # get the label
   ?o sc:name ?oname.
   ?p sc:name ?pname.
   FILTER (REGEX(?pname, ".*birth.*"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('author', 'http://www.wikidata.org/entity/Q1067'), ('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q2044'), ('oname', 'Florence')]
1


In [2]:

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   
   #set author as an instance of something to find birth date properties
   wd:Q1067 wdt:P31 ?obj.
   ?obj ?p ?o.
   
   
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   
   FILTER (REGEX(?oname, ".*date.*") )
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P570'), ('oname', 'date of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1636'), ('oname', 'date of baptism in early childhood')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oname', 'date of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P4602'), ('oname', 'date of burial or cremation')]
4


In [127]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('o', 'http://www.wikidata.org/entity/P569'), ('oname', 'date of birth')]


queryString = """
SELECT DISTINCT ?s ?sname ?date
WHERE {
   
   #Returning all poets from Florence with their birth date
   ?s wdt:P106 wd:Q49757 ;
      wdt:P19 wd:Q2044 ;
      wdt:P569 ?date.
   
   # get the label
   ?s sc:name ?sname.
   
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q17203201'), ('sname', 'Sebastiano Fenzi'), ('date', '1822-10-22T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q21003797'), ('sname', 'Michel Verin'), ('date', '1468-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q426305'), ('sname', 'Giovanni Maria Cecchi'), ('date', '1518-03-25T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1248954'), ('sname', 'Giovanni di Bernardo Rucellai'), ('date', '1475-10-29T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q7803'), ('sname', 'Bronzino'), ('date', '1503-11-27T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1067'), ('sname', 'Dante Alighieri'), ('date', '1265-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q177854'), ('sname', "Lorenzo de' Medici"), ('date', '1449-01-10T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q529858'), ('sname', 'Giovanni Papini'), ('date', '1881-01-09T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3694295'), ('sname', 'Cor

In [137]:

queryString = """
SELECT DISTINCT ?s ?sname ?date
WHERE {
   

   #Returns all poets from Florence with their birth date
   ?s wdt:P106 wd:Q49757 ;
      wdt:P19 wd:Q2044 ;
      wdt:P569 ?date.
    
    
      
   
   # get the label
   ?s sc:name ?sname.
   
   # Filtering reults
   FILTER (?date > "1299"^^xsd:gYear && ?date < "1501"^^xsd:gYear )
   
}
ORDER BY DESC (?date)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q19357719'), ('sname', 'Lodovico Martelli'), ('date', '1500-04-10T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3339583'), ('sname', 'Niccolò Martelli'), ('date', '1498-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q721846'), ('sname', 'Luigi Alamanni'), ('date', '1495-03-15T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q302218'), ('sname', 'Agnolo Firenzuola'), ('date', '1493-10-07T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q66809563'), ('sname', "Alessandro Pazzi de' Medici"), ('date', '1483-03-13T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1248954'), ('sname', 'Giovanni di Bernardo Rucellai'), ('date', '1475-10-29T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1158784'), ('sname', 'Crinitus'), ('date', '1475-01-18T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1215784'), ('sname', 'Alessandra Scala'), ('date', '1475-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1399'), 

Final query for this task

In [3]:

queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   

   #Returns all poets from Florence with their birth date
   ?s wdt:P106 wd:Q49757 ;
      wdt:P19 wd:Q2044 ;
      wdt:P569 ?date.
    
    
      
   
   # get the label
   ?s sc:name ?sname.
   
   # Filtering reults
   FILTER (?date > "1299"^^xsd:gYear && ?date < "1501"^^xsd:gYear )
   
}

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q62055718'), ('sname', "Lippo Pasci de' Bardi")]
[('s', 'http://www.wikidata.org/entity/Q1149308'), ('sname', 'Rustico Filippi')]
[('s', 'http://www.wikidata.org/entity/Q15633606'), ('sname', 'Bondie Dietaiuti')]
[('s', 'http://www.wikidata.org/entity/Q3763435'), ('sname', 'Gianni Alfani')]
[('s', 'http://www.wikidata.org/entity/Q366499'), ('sname', 'Orcagna')]
[('s', 'http://www.wikidata.org/entity/Q1402'), ('sname', 'Giovanni Boccaccio')]
[('s', 'http://www.wikidata.org/entity/Q604448'), ('sname', 'Antonio Pucci')]
[('s', 'http://www.wikidata.org/entity/Q50399164'), ('sname', "Dolcibene de' Tori")]
[('s', 'http://www.wikidata.org/entity/Q3677259'), ('sname', 'Cino Rinuccini')]
[('s', 'http://www.wikidata.org/entity/Q640594'), ('sname', 'John Dominici')]
[('s', 'http://www.wikidata.org/entity/Q45539848'), ('sname', 'Niccolò Tinucci')]
[('s', 'http://www.wikidata.org/entity/Q17166727'), ('sname', 'Antonio di Meglio')]
[('s', 'http://www.wi

## Task 5
Identify the BGP for literary work

In [148]:

queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   
   # Using The Divine Comedy (Q40185) as an example of a literary work and finding properties
   wd:Q40185 ?p ?obj .
   
   # get the label
   ?p sc:name ?pname .
   FILTER (!isLiteral(?obj))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P2551'), ('pname', 'used metre')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pname', 'language of work or name')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('pname', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P747'), ('pname', 'has edition or translation')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
12


In [150]:
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   
   
   wd:Q40185 wdt:P31 ?o .
   
   # get the label
   ?o sc:name ?oname .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q47461344'), ('oname', 'written work')]
1


In [154]:

queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   
   # Using Dante Alighieri (Q1067) as an example of poets and finding properties
   wd:Q1067 ?p ?obj .
   
   # get the label
   ?p sc:name ?pname .
   FILTER (!isLiteral(?obj))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('pname', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pname', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pname', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('pname', 'depicted by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('pname', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pname', 'ethnic group')]
[('p', 'http

In [157]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]

queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {
   
   # Using Dante Alighieri (Q1067) as an example of poets and finding properties
   wd:Q1067 wdt:P101 ?obj .
   
   # get the label
   ?obj sc:name ?oname .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q482'), ('oname', 'poetry')]
1


In [4]:

queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {
   
   #Using subclass property
   wd:Q482 wdt:P279 ?obj .
   
   # get the label
   ?obj sc:name ?oname .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7725634'), ('oname', 'literary work')]
1


Final query for this task

In [5]:

queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {
   
   #Using subclass property
   wd:Q482 wdt:P279 ?obj .
   
   # get the label
   ?obj sc:name ?oname .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7725634'), ('oname', 'literary work')]
1


## Task 6
Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

In [125]:

queryString = """
SELECT DISTINCT ?lwD ?literaryWork
WHERE {
   
   ?lwD wdt:P50 wd:Q1067 ;
        wdt:P407 ?lang .
   
   # get the label
   #?o sc:name ?oname.
   #?p sc:name ?pname.
   ?lwD sc:name ?literaryWork .
   ?lang sc:name ?language .
   
   FILTER (REGEX (?language , ".*[Ii]talian.*"))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('lwD', 'http://www.wikidata.org/entity/Q18084'), ('literaryWork', 'Vita Nuova')]
[('lwD', 'http://www.wikidata.org/entity/Q40185'), ('literaryWork', 'The Divine Comedy')]
[('lwD', 'http://www.wikidata.org/entity/Q3614562'), ('literaryWork', "Amore e 'l cor gentil sono una cosa")]
[('lwD', 'http://www.wikidata.org/entity/Q21191014'), ('literaryWork', 'Divina Commedia')]
[('lwD', 'http://www.wikidata.org/entity/Q666014'), ('literaryWork', 'Convivio')]
[('lwD', 'http://www.wikidata.org/entity/Q21191013'), ('literaryWork', 'Comedìa')]
[('lwD', 'http://www.wikidata.org/entity/Q1092462'), ('literaryWork', 'Le Rime')]
[('lwD', 'http://www.wikidata.org/entity/Q17562638'), ('literaryWork', 'Abandon all hope, ye who enter here')]
[('lwD', 'http://www.wikidata.org/entity/Q19222261'), ('literaryWork', 'Nulla mi parve mai più crudel cosa')]
[('lwD', 'http://www.wikidata.org/entity/Q19126932'), ('literaryWork', 'Commedia (1866, Lana)')]
[('lwD', 'http://www.wikidata.org/entity/Q3706230'), 

In [98]:
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work')]

queryString = """
SELECT DISTINCT ?lwD ?literaryWork
WHERE {
   
   wd:Q1067 wdt:P800 ?lwD .
        
   
   
   
   # get the label
   #?o sc:name ?oname.
   #?p sc:name ?pname.
   ?lwD sc:name ?literaryWork .

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('lwD', 'http://www.wikidata.org/entity/Q18084'), ('literaryWork', 'Vita Nuova')]
[('lwD', 'http://www.wikidata.org/entity/Q40185'), ('literaryWork', 'The Divine Comedy')]
[('lwD', 'http://www.wikidata.org/entity/Q134221'), ('literaryWork', 'De Monarchia')]
[('lwD', 'http://www.wikidata.org/entity/Q18081'), ('literaryWork', 'De vulgari eloquentia')]
[('lwD', 'http://www.wikidata.org/entity/Q666014'), ('literaryWork', 'Convivio')]
5


In [61]:

queryString = """
SELECT DISTINCT  ?pname  ?oname
WHERE {
   
   #Finding literary work properties
   wd:Q7725634 ?p ?o .
   
   # get the label
   ?o sc:name ?oname .
   ?p sc:name ?pname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pname', "topic's main template"), ('oname', 'Template:Infobox literary work')]
[('pname', 'different from'), ('oname', 'version, edition, or translation')]
[('pname', 'different from'), ('oname', 'written work')]
[('pname', 'properties for this type'), ('oname', 'GND ID')]
[('pname', 'properties for this type'), ('oname', 'author')]
[('pname', 'properties for this type'), ('oname', 'award received')]
[('pname', 'properties for this type'), ('oname', 'publisher')]
[('pname', 'properties for this type'), ('oname', 'set in period')]
[('pname', 'properties for this type'), ('oname', 'publication date')]
[('pname', 'properties for this type'), ('oname', 'has edition or translation')]
[('pname', 'properties for this type'), ('oname', 'country of origin')]
[('pname', 'properties for this type'), ('oname', 'narrative location')]
[('pname', 'properties for this type'), ('oname', 'language of work or name')]
[('pname', 'properties for this type'), ('oname', 'National Library of Israel

In [135]:
#From task 1: [('p', 'http://www.wikidata.org/prop/direct/P747'), ('name', 'has edition or translation')]

queryString = """
SELECT DISTINCT  ?p ?pname 
WHERE {
   
   #Finding literary work properties
   wd:Q40185 wdt:P747 ?o .
   ?o ?p ?obj .
   
   # get the label
   ?o sc:name ?oname .
   ?p sc:name ?pname .
   
   
   FILTER (REGEX (?pname, ".*translat.*"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P655'), ('pname', 'translator')]
[('p', 'http://www.wikidata.org/prop/direct/P747'), ('pname', 'has edition or translation')]
[('p', 'http://www.wikidata.org/prop/direct/P629'), ('pname', 'edition or translation of')]
3


In [144]:
#[('p', 'http://www.wikidata.org/prop/direct/P655'), ('pname', 'translator')]


queryString = """
SELECT DISTINCT ?lw ?literaryWorks 
WHERE {
   
   #Instances of literary works who were written by Dante A.
   ?lw wdt:P31 wd:Q7725634.
   ?lw wdt:P50 wd:Q1067.
       
   
   
   # get the label
   ?lw sc:name ?literaryWorks .
   #?o sc:name ?oname .
   #?p sc:name ?pname.
      
   #Since when we set a person as an author, also translation editions will be retrieved,
   #All literary works with translator property are excluded
   
   FILTER NOT EXISTS {?lw wdt:P655 ?t .}
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('lw', 'http://www.wikidata.org/entity/Q18084'), ('literaryWorks', 'Vita Nuova')]
[('lw', 'http://www.wikidata.org/entity/Q3798518'), ('literaryWorks', 'Fifth song of Dante\'s "Inferno"')]
[('lw', 'http://www.wikidata.org/entity/Q605018'), ('literaryWorks', 'Purgatorio')]
[('lw', 'http://www.wikidata.org/entity/Q134221'), ('literaryWorks', 'De Monarchia')]
[('lw', 'http://www.wikidata.org/entity/Q18081'), ('literaryWorks', 'De vulgari eloquentia')]
[('lw', 'http://www.wikidata.org/entity/Q666014'), ('literaryWorks', 'Convivio')]
[('lw', 'http://www.wikidata.org/entity/Q3730666'), ('literaryWorks', 'Epistles')]
[('lw', 'http://www.wikidata.org/entity/Q1092462'), ('literaryWorks', 'Le Rime')]
[('lw', 'http://www.wikidata.org/entity/Q3895042'), ('literaryWorks', 'Paradiso: Fifth Canto')]
[('lw', 'http://www.wikidata.org/entity/Q921273'), ('literaryWorks', 'Eclogues')]
[('lw', 'http://www.wikidata.org/entity/Q3926645'), ('literaryWorks', 'Quaestio de aqua et terra')]
11


In [141]:
queryString = """
SELECT DISTINCT ?poet ?poetName
WHERE {
   
  ?poet wdt:P106 wd:Q49757.
   
   
   
   # get the label
   ?poet sc:name ?poetName .
   
   #Finding Giovanni Boccaccio IRI
   FILTER (REGEX(?poetName, ".*[Bb]occaccio.*"))
    
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('poetName', 'Giovanni Boccaccio')]
1


In [145]:


queryString = """
SELECT DISTINCT ?lw ?literaryWorks 
WHERE {
   
   #Instances of literary works who were written by Giovanni B.
   ?lw wdt:P31 wd:Q7725634.
   ?lw wdt:P50 wd:Q1402.
       
   
   
   # get the label
   ?lw sc:name ?literaryWorks .
   #?o sc:name ?oname .
   #?p sc:name ?pname.
      
   #Since when we set a person as an author, also translation editions will be retrieved,
   #All literary works with translator property are excluded
   
   FILTER NOT EXISTS {?lw wdt:P655 ?t .}
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('lw', 'http://www.wikidata.org/entity/Q3745486'), ('literaryWorks', 'The Filocolo')]
[('lw', 'http://www.wikidata.org/entity/Q16438'), ('literaryWorks', 'The Decameron')]
[('lw', 'http://www.wikidata.org/entity/Q3614628'), ('literaryWorks', 'Amorosa visione')]
[('lw', 'http://www.wikidata.org/entity/Q3684237'), ('literaryWorks', 'Comedia delle ninfe fiorentine')]
[('lw', 'http://www.wikidata.org/entity/Q71736142'), ('literaryWorks', 'De Canaria')]
[('lw', 'http://www.wikidata.org/entity/Q3721314'), ('literaryWorks', 'Elegia di Madonna Fiammetta')]
[('lw', 'http://www.wikidata.org/entity/Q86673964'), ('literaryWorks', 'Mavortis Milex')]
[('lw', 'http://www.wikidata.org/entity/Q5244311'), ('literaryWorks', 'De casibus virorum illustrium')]
[('lw', 'http://www.wikidata.org/entity/Q3877086'), ('literaryWorks', 'Ninfale fiesolano')]
9


Final query for this task

In [143]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P655'), ('pname', 'translator')]


queryString = """
SELECT DISTINCT ?author ?authorName (COUNT(?lw) as ?literaryWorks)
WHERE {
   
   VALUES ?author { wd:Q1067 wd:Q1402 }
   #Instances of literary works who were written by authors
   ?lw wdt:P31 wd:Q7725634.
   ?lw wdt:P50 ?author .
       
   
   
   # get the label
   ?lw sc:name ?literaryWorks .
   ?author sc:name ?authorName .
  
      
   #Since when we set a person as an author, also translation editions will be retrieved,
   #All literary works with translator property are excluded
   
   FILTER NOT EXISTS {?lw wdt:P655 ?t .}
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('author', 'http://www.wikidata.org/entity/Q1402'), ('authorName', 'Giovanni Boccaccio'), ('literaryWorks', '9')]
[('author', 'http://www.wikidata.org/entity/Q1067'), ('authorName', 'Dante Alighieri'), ('literaryWorks', '11')]
2


## Task 7
Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

In [19]:

queryString = """
SELECT DISTINCT ?s ?sname ?cname
WHERE {
   
   #Returning all poets from Italy
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?cname.
   
   FILTER ( REGEX (?cname, ".*[Ii]taly.*"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)




Results
[('s', 'http://www.wikidata.org/entity/Q15726796'), ('sname', 'Teresa Albarelli'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q17203201'), ('sname', 'Sebastiano Fenzi'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q17285111'), ('sname', 'Francesca Manzoni'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q17453230'), ('sname', 'Maria Costa'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q17453230'), ('sname', 'Maria Costa'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q17453266'), ('sname', 'Gaetano Longo'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q17638293'), ('sname', 'Paolo Principato'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q22061622'), ('sname', 'Ernesto Del Preite'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q24206101'), ('sname', 'Federico Nardi'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/e

In [6]:

queryString = """
SELECT DISTINCT  ?p ?pname 
WHERE {
   
   #Returning all poets from Italy, find education characteristics 
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      ?p ?o .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?cname.
   ?o sc:name ?oname .
   ?p sc:name ?pname.
   
   #FILTER ( REGEX (?cname, ".*[Ii]taly.*"))
   FILTER ( REGEX (?pname, ".*[Aa]cademic.*"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P512'), ('pname', 'academic degree')]
[('p', 'http://www.wikidata.org/prop/direct/P812'), ('pname', 'academic major')]
[('p', 'http://www.wikidata.org/prop/direct/P1026'), ('pname', 'academic thesis')]
[('p', 'http://www.wikidata.org/prop/direct/P8413'), ('pname', 'academic appointment')]
4


In [43]:
#From results: ('p', 'http://www.wikidata.org/prop/direct/P512'), ('pname', 'academic degree')
#From Task 2: ('p', 'http://www.wikidata.org/prop/direct/P19'),('pname', 'place of birth')

queryString = """
SELECT DISTINCT  ?sname ?ciname ?dname
WHERE {
   
   #Returning all poets from Italy, their city of birth and education degree
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P19 ?city ;
      wdt:P512 ?degree .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?coname.
   ?city sc:name ?ciname.
   ?degree sc:name ?dname.
   
   FILTER ( REGEX (?coname, ".*[Ii]taly.*"))
}   
LIMIT 20
"""

print("Results")
x=run_query(queryString)




Results
[('sname', 'David Levi'), ('ciname', 'Chieri'), ('dname', 'laurea')]
[('sname', 'Luigi Mercantini'), ('ciname', 'Ripatransone'), ('dname', 'laurea')]
[('sname', 'Luigi Morandi'), ('ciname', 'Todi'), ('dname', 'laurea')]
[('sname', 'Vyacheslav Ivanovich Ivanov'), ('ciname', 'Moscow'), ('dname', 'Doctor of Sciences in Philology')]
[('sname', 'Boris Pahor'), ('ciname', 'Trieste'), ('dname', 'doctorate')]
[('sname', 'Giosuè Carducci'), ('ciname', 'Pietrasanta'), ('dname', 'laurea')]
[('sname', 'Tommaso Gnoli'), ('ciname', 'Rome'), ('dname', 'laurea')]
[('sname', 'Carlo Poerio'), ('ciname', 'Naples'), ('dname', 'laurea')]
[('sname', 'Nichi Vendola'), ('ciname', 'Bari'), ('dname', 'laurea')]
[('sname', 'Anselmo Guerrieri Gonzaga'), ('ciname', 'Mantua'), ('dname', 'laurea')]
[('sname', 'Nichi Vendola'), ('ciname', 'Terlizzi'), ('dname', 'laurea')]
[('sname', 'Olindo Guerrini'), ('ciname', 'Forlì'), ('dname', 'Legum Doctor')]
[('sname', 'Alojz Rebula'), ('ciname', 'San Pelagio'), ('dna

In [5]:

queryString = """
SELECT DISTINCT  ?p ?pname ?oname
WHERE {
   
   #Finding poetry subclasses
   ?s wdt:P106 wd:Q49757 ;
      ?p ?o .
   
   # get the label
   #?s sc:name ?sname .
   ?o sc:name ?oname .
   ?obj sc:name ?objname .
   ?p sc:name ?pname .
   
   FILTER ( REGEX (?pname, ".*educate*"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'Cornell University')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'University of Chicago')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'Brooklyn College')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'University of Havana')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'Johns Hopkins University')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'Palacký University Olomouc')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'Stanford University')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'University of Coimbra')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('oname', 'Pek

In [12]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at')

queryString = """
SELECT DISTINCT  ?sname ?ciname ?uniname 
WHERE {
   
   #Returning all poets from Italy, their city of birth and education degree
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P19 ?city ;
      wdt:P69 ?uni .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?coname.
   ?city sc:name ?ciname.
   ?uni sc:name ?uniname.
   
   FILTER ( REGEX (?coname, ".*[Ii]taly.*"))
}   
LIMIT 20
"""

print("Results")
x=run_query(queryString)




Results
[('sname', 'David Levi'), ('ciname', 'Chieri'), ('uniname', 'University of Pisa')]
[('sname', 'David Levi'), ('ciname', 'Chieri'), ('uniname', 'University of Parma')]
[('sname', 'Giorgio Bassani'), ('ciname', 'Bologna'), ('uniname', 'University of Bologna')]
[('sname', 'Angelo de Gubernatis'), ('ciname', 'Turin'), ('uniname', 'University of Turin')]
[('sname', 'Fabrizio De André'), ('ciname', 'Genoa'), ('uniname', 'University of Genoa')]
[('sname', 'Michelangelo Antonioni'), ('ciname', 'Ferrara'), ('uniname', 'University of Bologna')]
[('sname', 'Michelangelo Antonioni'), ('ciname', 'Ferrara'), ('uniname', 'Experimental Centre of Cinematography')]
[('sname', 'Alberto Bevilacqua'), ('ciname', 'Parma'), ('uniname', 'Romagnosi')]
[('sname', 'Renato Fucini'), ('ciname', 'Monterotondo Marittimo'), ('uniname', 'University of Pisa')]
[('sname', 'Muzi Epifani'), ('ciname', 'Benghazi'), ('uniname', 'Sapienza University of Rome')]
[('sname', 'Muzi Epifani'), ('ciname', 'Benghazi'), ('uni

In [25]:


queryString = """
SELECT DISTINCT  ?p ?pname 
WHERE {
   
   #Returning all poets from Italy, their city of birth and educated university
   #Trying to find university properties
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P19 ?city ;
      wdt:P69 ?uni .
   ?uni ?p ?o .
     
   
   # get the label
   #?s sc:name ?sname.
   #?country sc:name ?coname.
   #?city sc:name ?ciname.
   #?uni sc:name ?uniname.
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   
   FILTER ( REGEX (?pname, ".*[lL]ocation.*"))
}   
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P740'), ('pname', 'location of formation')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pname', 'location of discovery')]
[('p', 'http://www.wikidata.org/prop/direct/P1071'), ('pname', 'location of creation')]
[('p', 'http://www.wikidata.org/prop/direct/P937'), ('pname', 'work location')]
[('p', 'http://www.wikidata.org/prop/direct/P1740'), ('pname', 'category for films shot at this location')]
7


In [29]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location')]

queryString = """
SELECT DISTINCT  ?sname ?ciname ?uniname ?locname
WHERE {
   
   #Returning all poets from Italy, their city of birth , educated university and its location
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P19 ?city ;
      wdt:P69 ?uni .
    ?uni wdt:P159 ?location .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?coname.
   ?city sc:name ?ciname.
   ?uni sc:name ?uniname.
   ?location sc:name ?locname .
   
   FILTER ( REGEX (?coname, ".*[Ii]taly.*"))
}   
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('sname', 'David Levi'), ('ciname', 'Chieri'), ('uniname', 'University of Pisa'), ('locname', 'Pisa')]
[('sname', 'David Levi'), ('ciname', 'Chieri'), ('uniname', 'University of Parma'), ('locname', 'Parma')]
[('sname', 'Giorgio Bassani'), ('ciname', 'Bologna'), ('uniname', 'University of Bologna'), ('locname', 'Bologna')]
[('sname', 'Fabrizio De André'), ('ciname', 'Genoa'), ('uniname', 'University of Genoa'), ('locname', 'Genoa')]
[('sname', 'Michelangelo Antonioni'), ('ciname', 'Ferrara'), ('uniname', 'University of Bologna'), ('locname', 'Bologna')]
[('sname', 'Renato Fucini'), ('ciname', 'Monterotondo Marittimo'), ('uniname', 'University of Pisa'), ('locname', 'Pisa')]
[('sname', 'Muzi Epifani'), ('ciname', 'Benghazi'), ('uniname', 'Sapienza University of Rome'), ('locname', 'città universitaria of Rome')]
[('sname', 'Vittorio Sermonti'), ('ciname', 'Rome'), ('uniname', 'Sapienza University of Rome'), ('locname', 'città universitaria of Rome')]
[('sname', 'Renato Minore')

In [49]:


queryString = """
SELECT DISTINCT  ?city ?ciname ?sname
WHERE {
   
   #Returning all poets from Italy, their city of birth , educated university and its location
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P19 ?city ;
      wdt:P69 ?uni .
    ?uni wdt:P159 ?location .
    
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?coname.
   ?city sc:name ?ciname.
   ?uni sc:name ?uniname.
   ?location sc:name ?locname .
   
   FILTER ( REGEX (?coname, ".*[Ii]taly.*"))
   FILTER ( ?city = ?location )
} 

GROUP BY (?city)(?ciname) (?sname)

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('city', 'http://www.wikidata.org/entity/Q2044'), ('ciname', 'Florence'), ('sname', 'Fosco Maraini')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('ciname', 'Milan'), ('sname', 'Luciano Erba')]
[('city', 'http://www.wikidata.org/entity/Q1891'), ('ciname', 'Bologna'), ('sname', 'Pier Paolo Pasolini')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('ciname', 'Milan'), ('sname', 'Luigi Santucci')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('ciname', 'Milan'), ('sname', 'Ciro Fontana')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('ciname', 'Florence'), ('sname', 'Eugenio Miccini')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('ciname', 'Milan'), ('sname', 'Carlo Emilio Gadda')]
[('city', 'http://www.wikidata.org/entity/Q2683'), ('ciname', 'Parma'), ('sname', 'Attilio Bertolucci')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('ciname', 'Florence'), ('sname', 'Margherita Guidacci')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('ciname', '

Final query for this task

In [6]:


queryString = """
SELECT DISTINCT  ?city ?ciname (COUNT(DISTINCT ?s) as ?poets) 
WHERE {
   
   #Returning all poets from Italy, their city of birth , educated university and its location
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P19 ?city ;
      wdt:P69 ?uni .
    ?uni wdt:P159 ?location .
    
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?coname.
   ?city sc:name ?ciname.
   ?uni sc:name ?uniname.
   ?location sc:name ?locname .
   
   FILTER ( REGEX (?coname, ".*[Ii]taly.*"))
   FILTER ( ?city = ?location )
} 

GROUP BY (?city)(?ciname)
ORDER BY DESC (?poets)
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('city', 'http://www.wikidata.org/entity/Q490'), ('ciname', 'Milan'), ('poets', '10')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('ciname', 'Florence'), ('poets', '8')]
[('city', 'http://www.wikidata.org/entity/Q1449'), ('ciname', 'Genoa'), ('poets', '3')]
[('city', 'http://www.wikidata.org/entity/Q617'), ('ciname', 'Padua'), ('poets', '2')]
[('city', 'http://www.wikidata.org/entity/Q1891'), ('ciname', 'Bologna'), ('poets', '2')]
[('city', 'http://www.wikidata.org/entity/Q3519'), ('ciname', 'Bari'), ('poets', '1')]
[('city', 'http://www.wikidata.org/entity/Q2683'), ('ciname', 'Parma'), ('poets', '1')]
7


## Task 8
Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label).

In [59]:
queryString = """
SELECT DISTINCT ?s ?sname ?cname
WHERE {
   
   #Returning all poets from Italy
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?cname.
   
   FILTER ( REGEX (?cname, ".*[Ii]taly.*"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15726796'), ('sname', 'Teresa Albarelli'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q17203201'), ('sname', 'Sebastiano Fenzi'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q17285111'), ('sname', 'Francesca Manzoni'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q17453230'), ('sname', 'Maria Costa'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q17453230'), ('sname', 'Maria Costa'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q17453266'), ('sname', 'Gaetano Longo'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q17638293'), ('sname', 'Paolo Principato'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q22061622'), ('sname', 'Ernesto Del Preite'), ('cname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/entity/Q24206101'), ('sname', 'Federico Nardi'), ('cname', 'Italy')]
[('s', 'http://www.wikidata.org/e

In [64]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname  
WHERE {
   

   ?s wdt:P106 wd:Q49757 ;
      ?p ?o .
   
   # get the label
   ?o sc:name ?oname.
   ?p sc:name ?pname.
   
   FILTER ( REGEX (?pname, ".*award.*"))
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q59312531'), ('oname', 'Knight of the Order of Calatrava')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q3873143'), ('oname', "National Book Award for Young People's Literature")]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q1276342'), ('oname', 'Order of St. Sylvester')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q81466'), ('oname', 'Miguel de Cervantes Prize')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q20888812'), ('oname', 'Officer of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http:

In [ ]:
#('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')

queryString = """
SELECT DISTINCT ?award ?aname 
WHERE {
   

   ?s wdt:P106 wd:Q49757 ;
      wdt:P166 ?award .
   
   # get the label
   #?o sc:name ?oname.
   #?p sc:name ?pname.
   ?award sc:name ?aname .
   
   FILTER ( REGEX (?aname, ".*[Nn]obel.*"))
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Final query for this task

In [3]:
# [('award', 'http://www.wikidata.org/entity/Q37922'), ('aname', 'Nobel Prize in Literature')]
#('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')


queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
   
   #Returning all poets from Italy
   ?s wdt:P106 wd:Q49757 ;
      wdt:P27 ?country ;
      wdt:P166 wd:Q37922 .
     
   
   # get the label
   ?s sc:name ?sname.
   ?country sc:name ?cname.
   
   FILTER ( REGEX (?cname, ".*[Ii]taly.*"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q7728'), ('sname', 'Grazia Deledda')]
[('s', 'http://www.wikidata.org/entity/Q43440'), ('sname', 'Giosuè Carducci')]
[('s', 'http://www.wikidata.org/entity/Q1403'), ('sname', 'Luigi Pirandello')]
[('s', 'http://www.wikidata.org/entity/Q83038'), ('sname', 'Salvatore Quasimodo')]
[('s', 'http://www.wikidata.org/entity/Q83174'), ('sname', 'Eugenio Montale')]
[('s', 'http://www.wikidata.org/entity/Q765'), ('sname', 'Dario Fo')]
6
